Interactive version of the graph from [A/B Tests for Engineers](https://win-vector.com/2023/10/15/a-b-tests-for-engineers/).

To see the interaction: download [directory](https://github.com/WinVector/Examples/tree/main/ab_test) and run this Jupyter notebook in JupyterLab or VSCode.

[Jupyter Widgets documentation](https://ipywidgets.readthedocs.io/en/latest/index.html)

In [ ]:
# import our modules
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from sig_pow_visual import binomial_diff_sig_pow_visual, graph_factory
from data_algebra import RecordSpecification

import ipywidgets as widgets

In [ ]:
# our specification of interest
power = 0.9  # desired true positive rate for the test
significance = 0.02  # desired false positive rate for the test
# derived from the above
n = 557  # the experiment size
r = 0.1  # the assumed large effect size (difference in conversion rates)
t = 0.061576 # the correct threshold for specified power and significance


In [ ]:
mk_graphs = graph_factory(
    n=n,  # the experiment size
    r=r,  # the assumed large effect size (difference in conversion rates)
)

In [ ]:
menu = widgets.Dropdown(
    options = [0.0, 0.03, 0.061576, 0.09, 0.1],
    value = 0.061576,
    description='Decision threshold:',
    disabled=False
)

output = widgets.Output()
display(menu, output)

def on_year_selected(change):
    with output:
        if(menu.value != ''):
            clear_output(wait=True) 
            graphs = mk_graphs(menu.value)
            for g in graphs:
                print(g)
        else:
            clear_output(wait=False)
on_year_selected(menu.value)
menu.observe(on_year_selected, names='value')